In [1]:
import sys
import os
import gc
import time
import psutil
import numpy as np
import pandas as pd
import polars as pl
import warnings
from typing import Tuple, Dict, Any

warnings.filterwarnings("ignore")

try:
    from mars.feature.binning import MarsOptimalBinner, MarsNativeBinner
    from mars.utils.logger import set_log_level, logger
except ImportError:
    sys.path.append("./src") # 假设代码在 src 目录下
    from mars.feature.binning import MarsOptimalBinner, MarsNativeBinner
    from mars.utils.logger import set_log_level, logger

set_log_level("INFO")

def get_memory_usage() -> float:
    """获取当前进程内存占用 (MB)"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

print(f"✅ 环境就绪 | Polars: {pl.__version__} | Memory: {get_memory_usage():.2f} MB")


    __________________________________________________________________________
       __  ___ ___    ____  _____
      /  |/  //   |  / __ \/ ___/
     / /|_/ // /| | / /_/ /\__ \ 
    / /  / // ___ |/ _, _/___/ / 
   /_/  /_//_/  |_/_/ |_|/____/  
                                 
    MODELING ANALYSIS RISK SCORE 
    __________________________________________________________________________
    Version: 0.0.1 | Copyright (c) 2026 Christian Li
    High-performance Risk Modeling Toolkit powered by Polars
    __________________________________________________________________________

✅ 环境就绪 | Polars: 1.36.1 | Memory: 208.48 MB


In [2]:
def run_final_logic_tests():
    print("\n" + "="*60)
    print("🧪 模块 2: 最终版逻辑验证 (索引协议 & Join 路由)")
    print("="*60)

    # --- 1. 数值型：多特殊值与索引协议 ---
    print("\n[Case 1] 数值型：多特殊值索引验证")
    # 构造数据：含两个特殊值 -999, -998
    df_num = pl.DataFrame({"feat": [10, 20, -999, -998, None, 50]})
    y_num = [0, 0, 0, 0, 1, 1]
    
    binner = MarsOptimalBinner(
        features=["feat"], 
        n_bins=2, 
        special_values=[-999, -998],
        missing_values=[None]
    )
    binner.fit(df_num, y_num)
    
    # 验证 Index 模式
    res_idx = binner.transform(df_num, return_type="index")
    indices = res_idx["feat_bin"].to_list()
    print(f"   -> Indices: {indices}")
    # 协议验证：Missing=-1, Special1=-3, Special2=-4
    assert indices[4] == -1, "❌ Missing 索引错误"
    assert indices[2] == -3, "❌ 特殊值1 (-999) 索引错误"
    assert indices[3] == -4, "❌ 特殊值2 (-998) 索引错误"
    assert all(isinstance(x, int) for x in indices), "❌ 返回类型非整数"

    # --- 2. 类别型：高基数 Join 路由验证 ---
    print("\n[Case 2] 类别型：高基数 Join 路由测试")
    # 构造超过 join_threshold 的数据
    cats = [f"ID_{i}" for i in range(20)]
    df_cat = pl.DataFrame({"uid": cats})
    y_cat = np.random.randint(0, 2, 20)
    
    # 设置极低的阈值强制触发 Join
    binner_cat = MarsOptimalBinner(
        cat_features=["uid"], 
        n_bins=3, 
        join_threshold=5 
    )
    binner_cat.fit(df_cat, y_cat)
    
    # 验证 Join 后的索引输出
    res_cat = binner_cat.transform(df_cat, return_type="index")
    print(f"   -> Join Index Sample: {res_cat['uid_bin'].head(3).to_list()}")
    assert res_cat["uid_bin"].dtype in [pl.Int16, pl.Int8, pl.Int32], "❌ Join 路由输出类型错误"
    
    # 验证 Label 还原
    res_label = binner_cat.transform(df_cat, return_type="label")
    labels = res_label["uid_bin"].to_list()
    assert isinstance(labels[0], str) and "[" in labels[0], "❌ Join 路由 Label 还原错误"

    # --- 3. 类别型：Other 与 Special 瀑布流 ---
    print("\n[Case 3] 类别型：Other 与 Special 优先级")
    df_mix = pl.DataFrame({"city": ["A", "B", "C", "-999", None]})
    # 训练时只给 A, B
    df_train = pl.DataFrame({"city": ["A"]*10 + ["B"]*10})
    y_train = [1]*10 + [0]*10
    
    binner_mix = MarsOptimalBinner(
        cat_features=["city"], 
        special_values=["-999"]
    )
    binner_mix.fit(df_train, y_train)
    
    res_mix = binner_mix.transform(df_mix, return_type="index")
    mix_indices = res_mix["city_bin"].to_list()
    print(f"   -> Mix Indices: {mix_indices}")
    
    # 验证：A(0), B(1), C(Other:-2), -999(Special:-3), None(Missing:-1)
    assert mix_indices[2] == -2, "❌ 未见类别未归入 Other(-2)"
    assert mix_indices[3] == -3, "❌ 类别型特殊值映射错误"
    assert mix_indices[4] == -1, "❌ 类别型缺失值映射错误"

    # --- 4. 极致防御：类型转换稳定性 ---
    print("\n[Case 4] 类型防御：Int 列含空值")
    df_int = pl.DataFrame({"val": [1, 2, None]}, schema={"val": pl.Int64})
    binner_int = MarsOptimalBinner(features=["val"], n_bins=2)
    binner_int.fit(df_int, [0, 1, 0])
    # 验证不会因为 Int 列调用 is_nan 而崩溃
    try:
        binner_int.transform(df_int)
        print("   ✅ Int Null Safety 通关")
    except Exception as e:
        print(f"   ❌ Int Null Safety 失败: {e}")
        raise e
    
    # --- 5. 极致防御：混合类型配置抗毁性 ---
    print("\n[Case 5] 极致防御：混合类型配置抗毁性")
    print("   场景: 配置了字符串缺失值，但遇到了纯数值列。期望: 自动过滤，不报错。")
    
    df_safe = pl.DataFrame({"age": [20, 30, -999]}, schema={"age": pl.Int64})
    y_safe = [0, 1, 0]
    
    # 故意混入不兼容的类型 "unknown" 和 "N/A"
    binner_safe = MarsOptimalBinner(
        features=["age"], 
        n_bins=2, 
        missing_values=[-999, "unknown", "N/A"],
        special_values=[-1, "Error"]
    )
    
    try:
        binner_safe.fit(df_safe, y_safe)
        res_safe = binner_safe.transform(df_safe, return_type="index")
        
        # 验证 -999 是否正确被识别为 Missing (-1)
        # 如果 _get_safe_values 生效，"unknown" 应该被剔除，-999 保留
        idx_missing = res_safe.filter(pl.col("age") == -999)["age_bin"][0]
        assert idx_missing == -1, f"❌ 混合配置下数值缺失值未识别 (Got {idx_missing})"
        
        print("   ✅ Mixed Type Safety 通关")
    except Exception as e:
        print(f"   ❌ Mixed Type Safety 失败: {e}")
        raise e

    print("\n🎉 最终版逻辑验证全部通过！代码已达极致工业级水准。")

run_final_logic_tests()


🧪 模块 2: 最终版逻辑验证 (索引协议 & Join 路由)

[Case 1] 数值型：多特殊值索引验证
[MARS] 2026-01-15 23:53:14 - INFO - 📊 Features identified: 1 Numeric, 0 Categorical, 0 All-Null.
[MARS] 2026-01-15 23:53:14 - INFO - 🚀 [Numeric Pipeline] Starting Hybrid Engine for 1 features...
[MARS] 2026-01-15 23:53:14 - INFO -    [Stage 1] Pre-binning with Polars (Method: quantile, Pre-bins: 50)...
[MARS] 2026-01-15 23:53:14 - INFO - ⚙️ Fitting bins for 1 features (Native Mode: quantile)...
[MARS] 2026-01-15 23:53:14 - INFO - ⏱️ [MarsNativeBinner._fit_impl] finished in 0.0021s
[MARS] 2026-01-15 23:53:14 - INFO -    [Stage 1] Completed. 1 features passed to Solver.
[MARS] 2026-01-15 23:53:14 - INFO -    [Stage 2] Optimizing with Solver (Engine: cp, TimeLimit: 10s)...
[MARS] 2026-01-15 23:53:15 - INFO - ✅ [Numeric Pipeline] Optimization finished for 1 features.
   -> Indices: [0, 1, -3, -4, -1, 3]

[Case 2] 类别型：高基数 Join 路由测试
[MARS] 2026-01-15 23:53:15 - INFO - 📊 Features identified: 0 Numeric, 1 Categorical, 0 All-Null.
[MARS] 

In [ ]:
# ==========================================
# 模块 3: 大规模压力测试 (20万行 x 1000列 - 三模式对比)
# ==========================================

N_ROWS = 200_000
N_COLS = 1_000

def run_stress_test():
    print("\n" + "="*80)
    print(f"🔥 模块 3: 大规模压力测试 ({N_ROWS:,}行 x {N_COLS}列)")
    print("="*80)
    
    # 1. 生成海量数据 (Float32 节省内存)
    print(f"🚀 [DataGen] 正在生成数据...")
    data_matrix = np.random.randn(N_ROWS, N_COLS).astype(np.float32)
    y = (data_matrix[:, 0] > 0).astype(int)
    df = pl.from_numpy(data_matrix, schema=[f"f_{i}" for i in range(N_COLS)])
    print(f"✅ 数据准备完毕 | 内存占用: {get_memory_usage():.2f} MB")

    results = []
    methods = ["quantile", "uniform", "cart"]

    for method in methods:
        print(f"\n🧪 Testing Pre-binning Method: [{method.upper()}]")
        print("-" * 45)
        
        # 清理内存
        gc.collect()
        time.sleep(1) # 给系统回收缓冲的时间
        mem_start = get_memory_usage()
        t_start = time.time()
        
        # 2. 训练 MarsOptimalBinner
        # 混合动力引擎：Stage 1 (method) -> Stage 2 (CP Solver)
        binner = MarsOptimalBinner(
            n_bins=5, 
            n_prebins=50, 
            prebinning_method=method,
            n_jobs=-1,      
            time_limit=2    
        )
        
        binner.fit(df, y)
        t_fit = time.time() - t_start
        
        # 3. 转换 (默认返回 index 模式以测试极致性能)
        t_trans_start = time.time()
        res = binner.transform(df, return_type="index")
        # 强制触发 Polars 计算
        _ = res.select(pl.col("f_0_bin")).height 
        t_trans = time.time() - t_trans_start
        
        mem_peak = get_memory_usage() - mem_start
        
        results.append({
            "Method": method,
            "Fit Time(s)": t_fit,
            "Trans Time(s)": t_trans,
            "Mem Gain(MB)": mem_peak,
            "Throughput(f/s)": N_COLS / t_fit
        })
        
        print(f"   ⏱️  Fit Time:       {t_fit:.2f} s")
        print(f"   🚀  Trans Time:     {t_trans:.2f} s")
        print(f"   💾  Mem Overhead:   {mem_peak:.2f} MB")

    # 4. 打印汇总看板
    print("\n" + "🏆 STRESS TEST SUMMARY".center(60, "="))
    report_df = pd.DataFrame(results)
    display(report_df.round(3))
    
    # 彻底清理
    del df, data_matrix; gc.collect()

run_stress_test()


🔥 模块 3: 大规模压力测试 (200,000行 x 1000列)
🚀 [DataGen] 正在生成数据...
✅ 数据准备完毕 | 内存占用: 1717.12 MB

🧪 Testing Pre-binning Method: [QUANTILE]
---------------------------------------------


In [ ]:
from optbinning import BinningProcess

def benchmark_optimal_binning_with_check():
    # ==========================================
    # 1. 数据生成 (模拟大宽表场景)
    # ==========================================
    N_ROWS = 200_000
    N_COLS = 100
    
    print(f"🛠️ 正在生成测试数据: {N_ROWS} 行 x {N_COLS} 列...")
    np.random.seed(42)
    X = np.random.randn(N_ROWS, N_COLS).astype(np.float32)
    
    # 构造 Target: 简单的线性关系 + 噪声
    # 这种构造方式保证了特征具有一定的单调性趋势，利于 OptimalBinning 求解
    y = (X[:, 0] * 5 + X[:, 1] * 2 + np.random.randn(N_ROWS) > 0).astype(int)
    
    feat_names = [f"f_{i}" for i in range(N_COLS)]
    
    # 准备两种格式的数据
    df_pl = pl.from_numpy(X, schema=feat_names) # 给 Mars 用
    df_pd = pd.DataFrame(X, columns=feat_names) # 给 OptBinning 用
    
    print("✅ 数据准备完毕。\n")

    # ==========================================
    # 2. 定义测试配置
    # ==========================================
    methods = [
        "quantile", 
        "uniform", "cart"
        ]
    
    print(f"{'Method':<10} | {'Engine':<12} | {'Fit (s)':<10} | {'Trans (s)':<10} | {'Total (s)':<10} | {'Speedup':<8}")
    print("-" * 80)

    for method in methods:
        # ==========================================
        # Round 1: Mars (Hybrid Engine)
        # ==========================================
        gc.collect()
        t0 = time.time()
        
        mars_model = MarsOptimalBinner(
            n_bins=5, 
            n_prebins=50,       
            prebinning_method=method, 
            n_jobs=-1,
            time_limit=2        
        )
        
        mars_model.fit(df_pl, y)
        t_fit_end = time.time()
        
        res = mars_model.transform(df_pl)
        _ = res.select(pl.col(f"f_0_bin")).to_series() 
        t_trans_end = time.time()
        
        mars_fit_time = t_fit_end - t0
        mars_trans_time = t_trans_end - t_fit_end
        mars_total = mars_fit_time + mars_trans_time

        print(f"{method:<10} | {'Mars':<12} | {mars_fit_time:<10.4f} | {mars_trans_time:<10.4f} | {mars_total:<10.4f} | {'1.0x':<8}")

        # ==========================================
        # Round 2: OptBinning (Standard)
        # ==========================================
        gc.collect()
        t0 = time.time()
        
        # 1. 构造详细的参数配置字典
        fit_params = {}
        for col in feat_names:
            fit_params[col] = {
                # --- 预分箱参数 ---
                'prebinning_method': method, 
                'max_n_prebins': 50,
                
                # --- [关键] 核心优化参数 (对齐 Mars) ---
                # 强制 OptBinning 也进行双向搜索 (升序+降序)，而不是默认的猜方向
                'monotonic_trend': 'auto_asc_desc',  
                
                # 最小箱占比 (虽然后面 BinningProcess 也有这个参数，但写在这里优先级最高)
                'min_bin_size': 0.05
            }

        # 2. 初始化 BinningProcess
        opt_process = BinningProcess(
            variable_names=feat_names,
            max_n_bins=5,  
            n_jobs=-1,  
            binning_fit_params=fit_params 
        )
        
        opt_process.fit(df_pd, y)
        t_fit_end = time.time()
        
        _ = opt_process.transform(df_pd, metric="indices")
        t_trans_end = time.time()
        
        opt_fit_time = t_fit_end - t0
        opt_trans_time = t_trans_end - t_fit_end
        opt_total = opt_fit_time + opt_trans_time
        
        speedup = opt_total / mars_total
        print(f"{method:<10} | {'OptBinning':<12} | {opt_fit_time:<10.4f} | {opt_trans_time:<10.4f} | {opt_total:<10.4f} | {speedup:<8.1f}x")

        # ==========================================
        # 🔍 切点一致性抽检 (Check Splits) 冒烟测试 
        # ==========================================
        print(f"   [🔍 切点抽样对比: {method}]")
        
        # 随机抽取 3 个特征
        check_cols = np.random.choice(feat_names, 3, replace=False)
        
        for col in check_cols:
            # 1. 获取 Mars 切点 (去除 -inf, inf)
            if col in mars_model.bin_cuts_:
                mars_cuts = mars_model.bin_cuts_[col]
                # Mars 的格式是 [-inf, c1, c2, ..., inf]，比较时去掉无穷大
                mars_splits = [c for c in mars_cuts if c != float('-inf') and c != float('inf')]
            else:
                mars_splits = []

            # 2. 获取 OptBinning 切点
            # OptBinning 对象可能没有生成 splits (比如单调性约束无法满足)，需要处理异常
            try:
                opt_bin_obj = opt_process.get_binned_variable(col)
                if opt_bin_obj is not None:
                    opt_splits = opt_bin_obj.splits.tolist()
                else:
                    opt_splits = []
            except Exception:
                opt_splits = []
            
            # 3. 打印对比
            # 保留4位小数以便观察
            m_str = ", ".join([f"{x:.4f}" for x in mars_splits])
            o_str = ", ".join([f"{x:.4f}" for x in opt_splits])
            
            # 简单判断是否完全一致 (允许极小误差)
            is_close = False
            if len(mars_splits) == len(opt_splits):
                if len(mars_splits) == 0:
                    is_close = True
                else:
                    is_close = np.allclose(mars_splits, opt_splits, atol=1e-4)
            
            status_icon = "✅" if is_close else "⚠️"
            
            print(f"   - Feature {col:<5}: {status_icon}")
            print(f"     Mars: [{m_str}]")
            print(f"     Opt : [{o_str}]")
        
        print("-" * 80)

if __name__ == "__main__":
    benchmark_optimal_binning_with_check()